In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
from IPython.display import clear_output
import os

from quick_pp.objects import Project, Well

In [ ]:
folder = r"C:\projects\quick_pp\notebooks\data\01_raw"
file_names = os.listdir(folder)
files = [os.path.join(folder, file) for file in file_names]

In [ ]:
project = Project("Mock")
project.read_las(files)
clear_output()

In [ ]:
# # Load project from saved file
# project = Project().load(r"C:\projects\quick_pp\notebooks\data\04_project\test.qpp")

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_type
from quick_pp.plotter import plotly_log

In [ ]:
df = project.get_all_data()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
        dry_sand_point=(-.02, 2.65),
        dry_clay_point=(.33, 2.7),
        corr_angle=50
    )

    # Estimate lithology
    args = {
        # 'dry_clay_point': (0.27, 2.8)
        'silt_line_angle': 117,
        # 'wet_clay_point': (0.45, 2.45),
    }
    ssc_model = SandSiltClay(**args)
    vsand, vsilt, vcld, vclb, _ = ssc_model.estimate_lithology(
        nphihc, rhobhc, model='kuttan_modified', xplot=False, normalize= True
    )
    vclw = vcld + vclb

    # Estimate porosity
    phit = neu_den_xplot_poro(nphihc, rhobhc, model='ssc', reservoir=True,
                    dry_sand_point=(-.02, 2.65),
                    dry_silt_point=(.1, 2.68),
                    dry_clay_point=(.27, 2.7))

    rho_ma = rho_matrix(vsand, vsilt, vcld)
    phid = density_porosity(rhobhc, rho_ma)

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'] / 7, 'metric')  # Arbitrarily convert MD to TVD
    water_salinity = 30000
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    b = estimate_b_waxman_smits(temp_grad, rw)
    qv = estimate_qv(vcld, phit, cec_clay=.1)
    # TODO: Implement m estimation/ based on zonation
    m = 2

    swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv, b, m)
    swt_a = archie_saturation(well_data['RT'], rw, phit, 1, 2, 2)

    # Estimate permeability
    constant = vclb**1.75
    Swirr = constant / phit
    perm = choo_permeability(vclw, vsilt, phit)
    perm_kc = kozeny_carman_permeability(phit, Swirr)
    perm_t = timur_permeability(phit, Swirr)
    perm_c = coates_permeability(phit, Swirr)
    perm_tx = tixier_permeability(phit, Swirr)

    # Estimate rock types
    rock_flag = rock_type(vclw)

    # Estimate reservoir summary
    zones = well_data['ZONES'] = 'ALL' 
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=[0.3, 0.1, .75])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    well_data['VSAND'] = vsand
    well_data['VSILT'] = vsilt
    well_data['VCLD'] = vcld
    well_data['VCLW'] = vclw
    well_data['PHIT'] = phit
    well_data['PHID'] = phid
    well_data['SWT'] = swt
    well_data['PERM'] = perm
    well_data['ROCK_FLAG'] = rock_flag
    project.update_well_data(well_name, well_data)
    project.update_well_ressum(well_name, ressum_df)


In [ ]:
# Plot the results
well_name = 'X10_raw'
well_data = project.get_well_data(well_name)

fig = plotly_log(well_data)
fig.show()

In [ ]:
project.save('mock')